# Install Conda Environments in OpenSARlab using an environment.yml

**If you wish to create a conda environment with the conda create command, use this [notebook](../opensarlab_docs/OpenSARlab_supplements/Jupyter_Conda_Environments.ipynb) instead.**

In [1]:
import glob
import ipywidgets
import os
import fileinput

from IPython.display import Markdown, display

### List Currently Installed Conda Environments

In [2]:
!mamba env list

# conda environments:
#
hydrosar                 /home/jovyan/.local/envs/hydrosar
insar_analysis           /home/jovyan/.local/envs/insar_analysis
rtc_analysis             /home/jovyan/.local/envs/rtc_analysis
base                  *  /opt/conda



### Select a Conda Environment to Create

Note: 
to be seen by this notebook, new environment.yml files should be stored in /home/jovyan/Environment_Configs/

In [3]:
env_ymls = glob.glob("/home/jovyan/conda_environments/Environment_Configs/*.yml")

yml_dict = {fileinput.filename():f.split('name: ')[1].strip('\n') for f in fileinput.input(env_ymls) if fileinput.isfirstline()}
names = [v for v in yml_dict.values()]
if len(names) > len(set(names)):
    dups = {}
    
    for k, v in yml_dict.items():
        dups.setdefault(v, []).append(k)
    dups = {k:v for (k,v) in dups.items() if len(dups[k]) > 1}
    display(Markdown(f'<text style=color:red><strong>WARNING:</text>'))
    display(Markdown(f'<text style=color:red>Duplicate environment names found in /home/jovyan/conda_environments/Environment_Configs/*.yml files.</text>'))
    display(Markdown(f'<text style=color:red>Creating an env with the same name as an existing env will overwrite the original.</text>'))
    display(Markdown(f'<text style=color:green><br><strong>RECOMMENDATION:</strong></text>'))
    display(Markdown(f'<text style=color:green>Edit the environment names on the first lines of the YMLs to eliminate duplicates.</text>'))
    display(Markdown(f'<text style=color:green>If you see multiple similarly named YMLs, some of whose filenames have a timestamp appended to them, this is a result of YML updates.</text>'))
    display(Markdown(f'<text style=color:green>In this case, the YML without a timestamp is the latest version and, unless you have made customizations to the old versions, you may safely delete them.</text>'))
    
    for d in dups:
        print(f'YMLs using the environment name "{d}":')
        for pth in dups[d]:
            print(pth)
    options = [f"{yml_dict[k]} ---> {k}" for k in yml_dict]
else:
    options = [yml_dict[k] for k in yml_dict]

display(Markdown(f'<text style=color:green><br><strong>Select an Environment to Create:</strong></text>'))
height = len(env_ymls) * 20
names =  ipywidgets.RadioButtons(
    options=options,
    description="",
    disabled=False,
    layout=ipywidgets.Layout(height=f"{height}px", width='1000px')
)
names


<text style=color:green><br><strong>Select an Environment to Create:</strong></text>

RadioButtons(layout=Layout(height='160px', width='1000px'), options=('hydrosar', 'train', 'machine_learning', …

### Collect the Environment Name, Prefix, and Path to its env.yml

In [4]:
name = names.value
if '--->' in name:
    name, yml = name.split(' ---> ')
else:
    yml = {v:k for k,v in yml_dict.items()}[name]
prefix = f"/home/jovyan/.local/envs/{name}"

In [5]:
name

'unavco'

### Create the Environment, change its display name, and run any needed shell scripts
This will overwrite an existing enviornment with the same name instead of updating it, which is the safer option for environments that change PATH and/or PYTHONPATH

In [6]:
!mamba env create -f $yml --prefix $prefix -q --force

!mamba run -n $name kernda --display-name $name -o $prefix/share/jupyter/kernels/python3/kernel.json

if name == "insar_analysis":
    !source install_insar_analysis_pkgs.sh
elif name == "train":
    !source train.sh
elif name == "unavco":
    !source unavco.sh
elif name == "NISAR_SE":
    !source nisar_se.sh

  Package               Version  Build               Channel                    Size
──────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  conda_forge         conda-forge/linux-64     Cached
  + _openmp_mutex           4.5  2_gnu               conda-forge/linux-64     Cached
  + bzip2                 1.0.8  h7f98852_4          conda-forge/linux-64     Cached
  + ca-certificates   2022.9.24  ha878542_0          conda-forge/linux-64     Cached
  + kernda                0.3.0  py_1                conda-forge/noarch       Cached
  + ld_impl_linux-64     2.36.1  hea4e1c9_2          conda-forge/linux-64     Cached
  + libffi                3.4.2  h7f98852_5          conda-forge/linux-64     Cached
  + libgcc-ng            12.1.0  h8d9b700_16         conda-forge/linux-64     Cached
  + libgomp              12.1.0  h8d9b700_16     

### Remove unused packages and caches.

In [15]:
!mamba clean -p -t --yes


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.25.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████

Will remove 353 (621.3 MB) tarball(s).
Will remove 353 (2.28 GB) package(s).


*Create_OSL_Conda_Environments.ipynb - Version 1.2.0 - February 2022*

*Version Changes:*
- *conda -> mamba*